# Agenda:

* Data read and feature transformations
* Keras Model
* Keras Training loop

## Import Necessary Libraries

In [4]:
!pip install keras

In [10]:
import keras
print(keras.__version__)

2.13.1


In [12]:
!pip install tensorflow

  Obtaining dependency information for typing-extensions<4.6.0,>=3.6.6 from https://files.pythonhosted.org/packages/31/25/5abcd82372d3d4a3932e1fa8c3dbf9efac10cc7c0d16e78467460571b404/typing_extensions-4.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<2.0 from https://files.pythonhosted.org/packages/33/cf/8435d5a7159e2a9c83a95896ed596f68cf798005fe107cc655b5c5c14704/urllib3-1.26.20-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.0 MB/s eta 0:00:00
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 3.2 MB/s eta 0:00:0000:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.34 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
torch 2.4.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.
pydantic-core 2.23.3 requires typing-extensions!=4.7.0,>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
azure-storage-blob 12.23.1 requires typin

In [14]:
import tensorflow 
print(tensorflow.__version__)

2.13.0


In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

### Part 1: Import the Housing data and do feature transformations

In [16]:
df = pd.read_csv("house_price_full.csv")
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


#### Feature transformations

In [35]:
x = df.copy()

# Remove target from x and store in Y
Y = x.pop('price')

# Perform a scaler transform of the input data
scaler = StandardScaler()
x = scaler.fit_transform(x)

# Perform log transformations of target variable
Y = np.log(Y)

In [36]:
df_scaled = pd.DataFrame(x)
df_scaled

,0,1
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [37]:
Y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

### Part 3: Create Model Using `keras`

In [38]:
from tensorflow import keras

In [39]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            2, activation="sigmoid", input_shape=(x.shape[-1],)
        ),
        keras.layers.Dense(1, activation="linear")
    ]
)

In [40]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 2)                 6         
                                                                 
 dense_5 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9 (36.00 Byte)
Trainable params: 9 (36.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


```python
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2, 2)))
    b1 = tf.Variable(tf.random.uniform((1, 2)))
    w2 = tf.Variable(tf.random.uniform((2, 1)))
    b2 = tf.Variable(tf.random.uniform((1, 1)))
    return w1,b1,w2,b2


def forward_prop(x, w1, b1, w2, b2):
    z1 = tf.matmul(x,w1) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(h1,w2) + b2
    h2 = z2
    return h2
```

In [41]:
model.compile(
    optimizer=keras.optimizers.SGD(), loss="mean_squared_error"
)

```python
def train(x, y, w1, b1, w2, b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x, w1, b1, w2, b2)

        # loss
        loss = 0.5*(y_true - y_pred)** 2
    
    #Gradient calculation  
    print("**************************************************")
    print("GRADIENTS")
    print("**************************************************")
    gw1, gb1, gw2, gb2 = g.gradient(loss, [w1, b1, w2, b2])
    print(" the gradient for 1st layer weights are:\n",gw1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer weights are:\n",gw2.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 1st layer bias are:\n",gb1.numpy())
    print("--------------------------------------------------")
    print(" the gradient for 2nd layer bias are:\n",gb2.numpy())
    print("--------------------------------------------------")

    # Gradient descent:
    lr=0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1) 
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    print("**************************************************")
    print("NEW UPDATES")
    print("**************************************************")
    print(" the updated 1st layer weights are:\n",w1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer weights are:\n",w2.numpy())
    print("--------------------------------------------------")
    print(" the updated 1st layer bias are:\n",b1.numpy())
    print("--------------------------------------------------")
    print(" the updated 2nd layer bias are:\n",b2.numpy())


    return w1, b1, w2, b2,loss

```

In [42]:
model.fit(x, Y.values, epochs=10, batch_size=32)

Epoch 1/10
16/16 [==============================] - 0s 384us/step - loss: 119.0054
Epoch 2/10
16/16 [==============================] - 0s 353us/step - loss: 37.1616
Epoch 3/10
16/16 [==============================] - 0s 388us/step - loss: 9.0322
Epoch 4/10
16/16 [==============================] - 0s 365us/step - loss: 2.3245
Epoch 5/10
16/16 [==============================] - 0s 422us/step - loss: 0.9031
Epoch 6/10
16/16 [==============================] - 0s 340us/step - loss: 0.5513
Epoch 7/10
16/16 [==============================] - 0s 311us/step - loss: 0.4217
Epoch 8/10
16/16 [==============================] - 0s 350us/step - loss: 0.3454
Epoch 9/10
16/16 [==============================] - 0s 315us/step - loss: 0.2891
Epoch 10/10
16/16 [==============================] - 0s 358us/step - loss: 0.2478


In [43]:
model.predict(x)[:,0]

16/16 [==============================] - 0s 336us/step


array([12.906993 , 13.038776 , 13.21438  , 13.243156 , 12.96517  ,
       12.779192 , 13.109524 , 13.208235 , 13.389311 , 12.758008 ,
       13.113781 , 13.250605 , 13.359699 , 12.738794 , 12.944149 ,
       12.815687 , 13.0412855, 13.28212  , 12.925608 , 12.802044 ,
       13.330881 , 12.717457 , 13.133259 , 13.212643 , 12.802179 ,
       13.277987 , 13.368843 , 13.064938 , 12.9267025, 13.434225 ,
       12.780381 , 13.239353 , 12.91207  , 13.231483 , 13.341743 ,
       13.552755 , 12.714819 , 13.17962  , 13.226898 , 13.377762 ,
       13.046692 , 12.735056 , 12.755323 , 13.142796 , 13.019758 ,
       12.788252 , 13.231009 , 13.21438  , 12.862257 , 13.309229 ,
       13.208235 , 13.500559 , 13.459926 , 13.15685  , 12.761442 ,
       12.956643 , 13.061466 , 13.039571 , 12.894398 , 13.267605 ,
       13.309229 , 13.453164 , 13.212643 , 13.419119 , 13.250605 ,
       12.794922 , 13.083546 , 12.680027 , 13.644226 , 13.317385 ,
       13.164737 , 13.489149 , 13.091722 , 13.201584 , 12.9007